<a href="https://colab.research.google.com/github/worawit-saetan/Kaggle-Titanic-Dataset---Machine-Learning-from-Disaster/blob/main/Kaggle_Titanic_Dataset_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kaggle-Titanic-Dataset---Machine-Learning-from-Disaster

Predict survival on the Titanic and get familiar with ML basics

##The outline of our project

+ drop some unnecessary columns

+ fill in missing data

+ encode the data

+ train XGBoost model with GridSearchCV on training set

+ train another XGBoost model with best parameter setting (from gridsearch) on the merged training and test set

download the dataset to the program

In [1]:
import pandas as pd
dataset = pd.read_csv('/content/train.csv')

In [2]:
dataset.shape

(891, 12)

In [3]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


divide the dataset into X and y

In [4]:
X = dataset.drop(['Survived'],axis=1)

In [5]:
X.shape

(891, 11)

In [6]:
y = dataset['Survived']

In [7]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [8]:
y = pd.DataFrame(data=y,columns=['Survived'])

In [9]:
y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [10]:
y.shape

(891, 1)

split X and y to training and test set

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
X_train.shape

(712, 11)

In [14]:
X_test.shape

(179, 11)

In [15]:
y_train.shape

(712, 1)

In [16]:
y_test.shape

(179, 1)

Look at some information of the training set

In [17]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 778 to 579
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Name         712 non-null    object 
 3   Sex          712 non-null    object 
 4   Age          568 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Cabin        172 non-null    object 
 10  Embarked     710 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB


In [18]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
778,779,3,"Kilgannon, Mr. Thomas J",male,NaN,0,0,36865,7.7375,NaN,Q
879,880,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
302,303,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0000,NaN,S
700,701,1,"Astor, Mrs. John Jacob (Madeleine Talmadge Force)",female,18.0,1,0,PC 17757,227.5250,C62 C64,C
439,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S


Drop some columns that we think they are unnecessary

In [19]:
X_train = X_train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [20]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
778,3,male,NaN,0,0,7.7375,Q
879,1,female,56.0,0,1,83.1583,C
302,3,male,19.0,0,0,0.0000,S
700,1,female,18.0,1,0,227.5250,C
439,2,male,31.0,0,0,10.5000,S


In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 778 to 579
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       568 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


fill in missing data with median and mode options

In [22]:
from sklearn.impute  import SimpleImputer
import numpy as np

imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [23]:
numeric_columns = ['Pclass','Age','SibSp','Parch','Fare'] 

In [24]:
categorical_columns = ['Sex','Embarked']

In [25]:
imp_median.fit(X_train[numeric_columns])
imp_mode.fit(X_train[categorical_columns])

SimpleImputer(strategy='most_frequent')

In [26]:
X_train[numeric_columns] = imp_median.transform(X_train[numeric_columns])
X_train[categorical_columns] = imp_mode.transform(X_train[categorical_columns])

In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 778 to 579
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    float64
 1   Sex       712 non-null    object 
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    float64
 4   Parch     712 non-null    float64
 5   Fare      712 non-null    float64
 6   Embarked  712 non-null    object 
dtypes: float64(5), object(2)
memory usage: 44.5+ KB


Encode the data

In [28]:
from sklearn.preprocessing import LabelEncoder
label_enc_x = LabelEncoder()

label_enc_x.fit(X_train['Sex'])

LabelEncoder()

In [29]:
label_enc_x.classes_

array(['female', 'male'], dtype=object)

In [30]:
label_enc_x.transform(X_train['Sex'])

array([1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [31]:
X_train['Sex'] = label_enc_x.transform(X_train['Sex'])

In [32]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
778,3.0,1,28.0,0.0,0.0,7.7375,Q
879,1.0,0,56.0,0.0,1.0,83.1583,C
302,3.0,1,19.0,0.0,0.0,0.0000,S
700,1.0,0,18.0,1.0,0.0,227.5250,C
439,2.0,1,31.0,0.0,0.0,10.5000,S


One hot encoding the data

In [33]:
from sklearn.preprocessing import OneHotEncoder
onehot_enc = OneHotEncoder(handle_unknown='ignore')

# need to reshape(-1,1) because one hot encoder requires matrix not vector , and need to convert dataframe to array because function reshape(-1.1) will be available only for an array 
onehot_enc.fit(np.array(X_train['Embarked']).reshape(-1,1))

OneHotEncoder(handle_unknown='ignore')

In [34]:
onehot_enc.categories_


[array(['C', 'Q', 'S'], dtype=object)]

In [35]:
embarked_feature = onehot_enc.get_feature_names_out()
onehot_enc.get_feature_names_out()

array(['x0_C', 'x0_Q', 'x0_S'], dtype=object)

In [36]:
# need toarray() function to convert sparse matrix into an array
onehot_enc.transform(np.array(X_train['Embarked']).reshape(-1,1)).toarray()

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [37]:
X_train[embarked_feature] = onehot_enc.transform(np.array(X_train['Embarked']).reshape(-1,1)).toarray()


In [38]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,x0_C,x0_Q,x0_S
778,3.0,1,28.0,0.0,0.0,7.7375,Q,0.0,1.0,0.0
879,1.0,0,56.0,0.0,1.0,83.1583,C,1.0,0.0,0.0
302,3.0,1,19.0,0.0,0.0,0.0000,S,0.0,0.0,1.0
700,1.0,0,18.0,1.0,0.0,227.5250,C,1.0,0.0,0.0
439,2.0,1,31.0,0.0,0.0,10.5000,S,0.0,0.0,1.0


In [39]:
X_train.drop(["Embarked"],axis=1,inplace=True)

In [40]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,x0_C,x0_Q,x0_S
778,3.0,1,28.0,0.0,0.0,7.7375,0.0,1.0,0.0
879,1.0,0,56.0,0.0,1.0,83.1583,1.0,0.0,0.0
302,3.0,1,19.0,0.0,0.0,0.0000,0.0,0.0,1.0
700,1.0,0,18.0,1.0,0.0,227.5250,1.0,0.0,0.0
439,2.0,1,31.0,0.0,0.0,10.5000,0.0,0.0,1.0


In [41]:
X_train.dtypes

Pclass    float64
Sex         int64
Age       float64
SibSp     float64
Parch     float64
Fare      float64
x0_C      float64
x0_Q      float64
x0_S      float64
dtype: object

Create the XGBoost model and to do the gridsearch option to find the best parameter settings.

In [51]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

params_xgb = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(100, 210, 10),
    'learning_rate': [0.1, 0.01, 0.05]
}

gs_xgb = GridSearchCV(estimator=xgb , param_grid=params_xgb, cv=5, verbose=3, scoring='accuracy')

the model require y_train to be an array. let's change it

In [54]:
y_train

,Survived
778,0
879,1
302,0
700,1
439,0
...,...
290,1
412,1
803,1
324,0


In [55]:
y_train = y_train['Survived']

In [56]:
y_train

778    0
879    1
302    0
700    1
439    0
      ..
290    1
412    1
803    1
324    0
579    1
Name: Survived, Length: 712, dtype: int64

Train the model with training data

In [57]:
gs_xgb.fit(X_train,y_train)

Fitting 5 folds for each of 264 candidates, totalling 1320 fits
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.804 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.797 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.845 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.810 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.810 total time=   0.1s
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.790 total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.804 total time=   0.1s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.845 total time=   0.1s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.817 total time=   0.1s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=110;, score=0.824 total

GridSearchCV(cv=5, estimator=XGBClassifier(nthread=4, seed=42),
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(100, 210, 10)},
             scoring='accuracy', verbose=3)

Look at the best parameter setting and its score (accuracy)

In [58]:
gs_xgb.best_params_

{'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 100}

In [59]:
gs_xgb.best_score_

0.8287304245050724

Let's evaluate the test data but we need to do some feature engineering steps. the same process as we do on the training data

In [60]:
X_test = X_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

X_test[numeric_columns] = imp_median.transform(X_test[numeric_columns])
X_test[categorical_columns] = imp_mode.transform(X_test[categorical_columns])

X_test['Sex'] = label_enc_x.transform(X_test['Sex'])

X_test[embarked_feature] = onehot_enc.transform(np.array(X_test['Embarked']).reshape(-1,1)).toarray()

X_test.drop(["Embarked"],axis=1,inplace=True)

y_test = y_test['Survived']

In [61]:
y_test

421    0
221    0
446    1
128    1
109    1
      ..
413    0
704    0
758    0
258    1
662    0
Name: Survived, Length: 179, dtype: int64

Check the score on training data

In [64]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_train,gs_xgb.predict(X_train))


0.8876404494382022

Check the score on test data

In [66]:
accuracy_score(y_test,gs_xgb.predict(X_test))

0.8547486033519553

In [71]:
gs_xgb.predict(X_test)

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0])

Let''s load the kaggle test data (without target y)

In [72]:
import pandas as pd
X_realworld = pd.read_csv('/content/test.csv')

In [73]:
X_realworld

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Do the feature engineering steps to the kaggle test data

In [74]:
X_realworld = X_realworld.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

X_realworld[numeric_columns] = imp_median.transform(X_realworld[numeric_columns])
X_realworld[categorical_columns] = imp_mode.transform(X_realworld[categorical_columns])

X_realworld['Sex'] = label_enc_x.transform(X_realworld['Sex'])

X_realworld[embarked_feature] = onehot_enc.transform(np.array(X_realworld['Embarked']).reshape(-1,1)).toarray()

X_realworld.drop(["Embarked"],axis=1,inplace=True)

In [75]:
X_realworld

,Pclass,Sex,Age,SibSp,Parch,Fare,x0_C,x0_Q,x0_S
0,3.0,1,34.5,0.0,0.0,7.8292,0.0,1.0,0.0
1,3.0,0,47.0,1.0,0.0,7.0000,0.0,0.0,1.0
2,2.0,1,62.0,0.0,0.0,9.6875,0.0,1.0,0.0
3,3.0,1,27.0,0.0,0.0,8.6625,0.0,0.0,1.0
4,3.0,0,22.0,1.0,1.0,12.2875,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
413,3.0,1,28.0,0.0,0.0,8.0500,0.0,0.0,1.0
414,1.0,0,39.0,0.0,0.0,108.9000,1.0,0.0,0.0
415,3.0,1,38.5,0.0,0.0,7.2500,0.0,0.0,1.0
416,3.0,1,28.0,0.0,0.0,8.0500,0.0,0.0,1.0


Lest's predict the kaggle data

In [76]:
gs_xgb.predict(X_realworld)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Load the submisssion file to create our own prediction file (using our model)

In [77]:
gender_submission = pd.read_csv('/content/gender_submission.csv')

In [78]:
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [79]:
gs_xgb_submission = gender_submission.copy()

In [81]:
gs_xgb_submission['Survived'] = gs_xgb.predict(X_realworld)
gs_xgb_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


Save the excel file, then finished

In [82]:
gs_xgb_submission.to_csv('/content/gs_xgb_submission.csv',index=False)

Lets try another approach that is training our model on both training and test  set

In [92]:
X_train_test = pd.concat([X_train, X_test], ignore_index=False, sort=False)

In [93]:
y_train_test = pd.concat([y_train,y_test])

our model with best parameter setting

In [96]:
xgb_best_setting_from_gs = XGBClassifier(
    objective = 'binary:logistic',
    nthread = 4,
    seed = 42,
    learning_rate = 0.01,
    max_depth = 8,
    n_estimators = 100
)


train our model

In [97]:
xgb_best_setting_from_gs.fit(X_train_test,y_train_test)

XGBClassifier(learning_rate=0.01, max_depth=8, nthread=4, seed=42)

take our predictions of our model into excel format

In [98]:
gs_xgb_merge_train_test_submission = gender_submission.copy()

gs_xgb_merge_train_test_submission['Survived'] = xgb_best_setting_from_gs.predict(X_realworld)
gs_xgb_merge_train_test_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [99]:
gs_xgb_merge_train_test_submission.to_csv('/content/gs_xgb_merge_train_test_submission.csv',index=False)